In [ ]:
import networkx as nx
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import zipfile
import requests
import io
import time

In [ ]:
def download_and_read_data():
    """
    Tải và đọc dữ liệu mạng các trang Facebook về thực phẩm
    Returns:
        DataFrame: Chứa các cạnh của đồ thị
    """
    print("Đang tải dữ liệu Facebook Food Pages Network...")

    # URL của dataset
    url = "https://nrvis.com/download/data/soc/fb-pages-food.zip"

    try:
        # Tải dữ liệu
        response = requests.get(url)
        if response.status_code == 200:
            # Đọc file zip
            z = zipfile.ZipFile(io.BytesIO(response.content))

            # Đọc dữ liệu từ file trong zip
            df = pd.read_csv(z.open('fb-pages-food.edges'),
                           sep=' ',
                           names=['source', 'target'])
            return df
        else:
            raise Exception("Không thể tải dữ liệu")
    except Exception as e:
        print(f"Lỗi khi tải dữ liệu: {e}")
        return None

In [ ]:
def analyze_network(G):
    """
    Phân tích đặc trưng của mạng các trang Facebook
    Args:
        G: Đồ thị NetworkX
    Returns:
        dict: Các số đo của mạng
    """
    metrics = {}

    # 1. Thông tin cơ bản
    print("\nĐang tính toán thông tin cơ bản...")
    metrics['nodes'] = G.number_of_nodes()  # Số lượng trang
    metrics['edges'] = G.number_of_edges()  # Số lượng liên kết
    metrics['density'] = nx.density(G)      # Mật độ liên kết

    # 2. Degree Analysis
    print("Đang tính toán degree metrics...")
    start_time = time.time()
    degrees = dict(G.degree())
    metrics['avg_degree'] = np.mean(list(degrees.values()))
    metrics['max_degree'] = max(degrees.values())
    metrics['degrees'] = degrees
    print(f"Thời gian tính Degree metrics: {time.time() - start_time:.2f}s")

    # 3. Degree Centrality
    print("Đang tính toán Degree Centrality...")
    start_time = time.time()
    dc = nx.degree_centrality(G)
    metrics['degree_centrality'] = {
        'values': dc,
        'max': max(dc.values()),
        'avg': sum(dc.values()) / len(dc),
        'top_pages': sorted(dc.items(), key=lambda x: x[1], reverse=True)[:10]
    }
    print(f"Thời gian tính Degree Centrality: {time.time() - start_time:.2f}s")

    # 4. Betweenness Centrality
    print("Đang tính toán Betweenness Centrality...")
    start_time = time.time()
    bc = nx.betweenness_centrality(G)
    metrics['betweenness_centrality'] = {
        'values': bc,
        'max': max(bc.values()),
        'avg': sum(bc.values()) / len(bc),
        'top_pages': sorted(bc.items(), key=lambda x: x[1], reverse=True)[:10]
    }
    print(f"Thời gian tính Betweenness Centrality: {time.time() - start_time:.2f}s")

    # 5. Closeness Centrality
    print("Đang tính toán Closeness Centrality...")
    start_time = time.time()
    cc = nx.closeness_centrality(G)
    metrics['closeness_centrality'] = {
        'values': cc,
        'max': max(cc.values()),
        'avg': sum(cc.values()) / len(cc),
        'top_pages': sorted(cc.items(), key=lambda x: x[1], reverse=True)[:10]
    }
    print(f"Thời gian tính Closeness Centrality: {time.time() - start_time:.2f}s")

    # 6. PageRank
    print("Đang tính toán PageRank...")
    start_time = time.time()
    pr = nx.pagerank(G, alpha=0.85)
    metrics['pagerank'] = {
        'values': pr,
        'max': max(pr.values()),
        'avg': sum(pr.values()) / len(pr),
        'top_pages': sorted(pr.items(), key=lambda x: x[1], reverse=True)[:10]
    }
    print(f"Thời gian tính PageRank: {time.time() - start_time:.2f}s")

    return metrics

In [ ]:
def print_results(metrics):
    """
    In kết quả phân tích
    """
    print("\nKẾT QUẢ PHÂN TÍCH MẠNG FACEBOOK FOOD PAGES")
    print("=" * 50)

    print("\n1. THÔNG TIN CƠ BẢN")
    print("-" * 30)
    print(f"Số lượng trang: {metrics['nodes']:,}")
    print(f"Số lượng liên kết: {metrics['edges']:,}")
    print(f"Mật độ mạng: {metrics['density']:.6f}")
    print(f"Số liên kết trung bình/trang: {metrics['avg_degree']:.2f}")
    print(f"Số liên kết nhiều nhất: {metrics['max_degree']}")

    print("\n2. PHÂN TÍCH CENTRALITY")
    centrality_measures = {
        'Degree Centrality': 'degree_centrality',
        'Betweenness Centrality': 'betweenness_centrality',
        'Closeness Centrality': 'closeness_centrality',
        'PageRank': 'pagerank'
    }

    for name, measure in centrality_measures.items():
        print(f"\n{name}:")
        print(f"- Giá trị trung bình: {metrics[measure]['avg']:.6f}")
        print(f"- Giá trị lớn nhất: {metrics[measure]['max']:.6f}")
        print("\nTop 5 trang có giá trị cao nhất:")
        for page, score in metrics[measure]['top_pages'][:5]:
            print(f"Page {page}: {score:.6f}")


In [ ]:
# Tải dữ liệu
edges_df = download_and_read_data()

# Tạo đồ thị có hướng
print("Đang tạo đồ thị...")
G = nx.from_pandas_edgelist(edges_df, 'source', 'target', create_using=nx.DiGraph())

In [ ]:
# Phân tích mạng
metrics = analyze_network(G)

In [ ]:
# In kết quả
print_results(metrics)